In [1]:
!pip install langchain==0.0.270
!pip install faiss-cpu langchain==0.0.271 openai tiktoken

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.5 MB 1.9 MB/s eta 0:00:01
   -- ------------------------------------- 0.1/1.5 MB 1.1 MB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.5 MB 1.5 MB/s eta 0:00:01
   --------- ------------------------------ 0.4/1.5 MB 2.2 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.5 MB 2.6 MB/s eta 0:00:01
   ------------------ --------------------- 0.7/1.5 MB 3.2 MB/s eta 0:00:01
   ------------------------ --------------- 0.9/1.5 MB 3.1 MB/s eta 0:00:01
   -------------------------------- ------- 1.3/1.5 MB 3.6 MB/s eta 0:00:01
   -------------------------------- ------- 1.3/1.5 MB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/56.5 kB ? eta -:--:--
   ---------------------------------------- 56.5/56.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.6 requires langchain<0.3.0,>=0.2.6, but you have langchain 0.0.270 which is incompatible.
langchain-community 0.2.6 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.
langchain-core 0.2.11 requires langsmith<0.2.0,>=0.1.75, but you have langsmith 0.0.92 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 435.7 kB/s eta 0:00:04
   - -------------------------------------- 0.1/1.5 MB 648.1 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.5 MB 944.1 kB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.5 MB 1.7 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.5 MB 2.3 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.5 MB 2.9 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.5 MB 2.9 MB/s eta 0:00:01
   ------------------------------------ --- 1.4/1.5 MB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 3.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   - -------------------------------------- 0.6/14.6 MB 12.0 MB/s eta 0:00:02
   -- ------------------------------------- 1.0/14.6 MB 10.4 MB/s eta 0:00:02
   --- --------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.6 requires langchain<0.3.0,>=0.2.6, but you have langchain 0.0.271 which is incompatible.
langchain-community 0.2.6 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests
import openai
from langchain.docstore.document import Document

In [4]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [5]:
# База знаний, которая будет подаваться в langChain
data_from_url= load_document_text('https://docs.google.com/document/d/1ova0gprJGQ7_wYJhxEozpzZTgsGYYgyADeNOm-l7rfM')


In [6]:
data_from_url

'\ufeffКАСКО в Simble теперь можно приобрести в рассрочку с картой Свобода\\n\\nТеперь КАСКО в Simble можно приобрести в рассрочку с картой Свобода от Банка Хоум Кредит. Карта Свобода предоставляет возможность делать покупки в рассрочку до 12 месяцев у партнеров или до 51 дня в любых магазинах. С помощью карты Свобода можно оформить КАСКО на короткий срок в приложении Simble. Узнать подробнее об условиях предложения и оформить карту Свобода можно [здесь](ссылка).\r\n\r\n\r\nSimble – КАСКО на короткий период\\n\\nSimble предлагает страховку на короткий период, которая позволяет застраховать автомобиль на 1 час поездки, 100 км пути или на время парковки. Стоимость страховки начинается от 14 ₽ в день. Также в приложении Simble можно оформить КАСКО на год поездки или парковку. Страховка на парковку предлагается по часам, начиная от 13 ₽ в час. Страховка на поездку предлагается по часам, начиная от 95 ₽ в час, или на любое количество километров, начиная от 175 ₽ за 100 км. Simble также пред

In [24]:
# Открываем файл для чтения
with open('C:/Users/lenovo/Desktop/ITE Group/Telegram_Bot/.venv/Simble база знаний.txt', 'r') as file:
    # Читаем содержимое файла и сохраняем его в переменную content
    content = file.read()
    # Выводим содержимое файла на экран
    print(content)

КАСКО в Simble теперь можно приобрести в рассрочку с картой Свобода\n\nТеперь КАСКО в Simble можно приобрести в рассрочку с картой Свобода от Банка Хоум Кредит. Карта Свобода предоставляет возможность делать покупки в рассрочку до 12 месяцев у партнеров или до 51 дня в любых магазинах. С помощью карты Свобода можно оформить КАСКО на короткий срок в приложении Simble. Узнать подробнее об условиях предложения и оформить карту Свобода можно [здесь](ссылка).


Simble – КАСКО на короткий период\n\nSimble предлагает страховку на короткий период, которая позволяет застраховать автомобиль на 1 час поездки, 100 км пути или на время парковки. Стоимость страховки начинается от 14 ₽ в день. Также в приложении Simble можно оформить КАСКО на год поездки или парковку. Страховка на парковку предлагается по часам, начиная от 13 ₽ в час. Страховка на поездку предлагается по часам, начиная от 95 ₽ в час, или на любое количество километров, начиная от 175 ₽ за 100 км. Simble также предлагает тарифы для ра

In [25]:
source_chunks = []
splitter = CharacterTextSplitter(separator=" ", chunk_size=102, chunk_overlap=0)

for chunk in splitter.split_text(content):
    source_chunks.append(Document(page_content=chunk, metadata={}))

In [26]:
# всего получилось чанков:
len(source_chunks)

983

In [27]:
# первый чанк
print(source_chunks[0])
page_content = source_chunks[0].page_content
# длина первого чанка
print(len(page_content))

page_content='\ufeffКАСКО в Simble теперь можно приобрести в рассрочку с картой Свобода\\n\\nТеперь КАСКО в Simble можно' metadata={}
99


In [28]:
# Устанавливаем метаданные для первого чанка
source_chunks[0].metadata = {"teg": "раздел 1"}

# выводим на печать обновленные метаданные чанка
print(source_chunks[0].metadata)

{'teg': 'раздел 1'}


In [30]:
loader = TextLoader("C:/Users/lenovo/Desktop/ITE Group/Telegram_Bot/.venv/Simble база знаний.txt")

data_from_file = loader.load()

In [31]:
data_from_file[0]

Document(page_content='\ufeffКАСКО в Simble теперь можно приобрести в рассрочку с картой Свобода\\n\\nТеперь КАСКО в Simble можно приобрести в рассрочку с картой Свобода от Банка Хоум Кредит. Карта Свобода предоставляет возможность делать покупки в рассрочку до 12 месяцев у партнеров или до 51 дня в любых магазинах. С помощью карты Свобода можно оформить КАСКО на короткий срок в приложении Simble. Узнать подробнее об условиях предложения и оформить карту Свобода можно [здесь](ссылка).\n\n\nSimble – КАСКО на короткий период\\n\\nSimble предлагает страховку на короткий период, которая позволяет застраховать автомобиль на 1 час поездки, 100 км пути или на время парковки. Стоимость страховки начинается от 14 ₽ в день. Также в приложении Simble можно оформить КАСКО на год поездки или парковку. Страховка на парковку предлагается по часам, начиная от 13 ₽ в час. Страховка на поездку предлагается по часам, начиная от 95 ₽ в час, или на любое количество километров, начиная от 175 ₽ за 100 км. S

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [33]:
source_chunks=[]
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
for chunk in splitter.split_text(content):
    source_chunks.append(Document(page_content=chunk, metadata={}))
len(source_chunks)

97

In [34]:
source_chunks[2]

Document(page_content='Как оформить страховку в Simble\nЧтобы оформить страховку в Simble, необходимо скачать приложение на устройство с операционной системой iOS или Android. Затем нужно указать свои данные и данные автомобиля. Если автомобиль уже зарегистрирован, можно найти его по номеру, если нет, то нужно указать данные ПТС. После этого необходимо сфотографировать автомобиль со всех сторон. Фотоосмотр нужно пройти только один раз, при первой покупке страховки. После фотоосмотра страховка будет одобрена в течение примерно 3 часов. Затем можно выбрать тариф и купить полис. \n# КАСКО и ОСАГО в Simble\nSimble также предлагает возможность сравнить цены на ОСАГО от 16 страховых компаний и оформить полис за 10 минут без заполнения бумаг. Все цены на ОСАГО на сайте Simble являются прозрачными и без скрытых комиссий и надбавок. Simble также предоставляет информацию о том, что покрывает полис ОСАГО и какие риски он покрывает.', metadata={})

In [35]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()



In [38]:
import openai

In [40]:
# Создадим индексную базу из разделенных фрагментов текста
try:
    
    db = FAISS.from_documents(source_chunks, embeddings)
    
except Exception as e:
    print("Произошла ошибка:", e)

Произошла ошибка: module 'openai' has no attribute 'error'


In [46]:
from openai import error

ImportError: cannot import name 'error' from 'openai' (c:\Users\lenovo\anaconda3\Lib\site-packages\openai\__init__.py)

In [45]:
pip install openai --upgrade

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(source_chunks, embeddings)

AttributeError: module 'openai' has no attribute 'error'